In [24]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [25]:
!pip install webdriver-manager

In [26]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

In [45]:
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import re
import time

window_path = "chromedriver.exe"

source_url = "https://map.kakao.com/"

driver = webdriver.Chrome(window_path)
driver.get(source_url)

searchbox = driver.find_element(By.XPATH, "//input[@id='search.keyword.query']")
searchbox.send_keys("강남역 피자")

searchbutton = driver.find_element(By.XPATH, "//button[@id='search.keyword.submit']")
driver.execute_script("arguments[0].click();", searchbutton)

time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

moreviews = soup.find_all(name="a", attrs={"class":"moreview"})
page_urls = []
for moreview in moreviews:
    page_url = moreview.get("href")
    print(page_url)
    page_urls.append(page_url)

driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.92)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x01188893+48451]
	(No symbol) [0x0111B8A1]
	(No symbol) [0x01025058]
	(No symbol) [0x0100D073]
	(No symbol) [0x0106DEBB]
	(No symbol) [0x0107BFD3]
	(No symbol) [0x0106A0B6]
	(No symbol) [0x01047E08]
	(No symbol) [0x01048F2D]
	GetHandleVerifier [0x013E8E3A+2540266]
	GetHandleVerifier [0x01428959+2801161]
	GetHandleVerifier [0x0142295C+2776588]
	GetHandleVerifier [0x01212280+612144]
	(No symbol) [0x01124F6C]
	(No symbol) [0x011211D8]
	(No symbol) [0x011212BB]
	(No symbol) [0x01114857]
	BaseThreadInitThunk [0x77190099+25]
	RtlGetAppContainerNamedObjectPath [0x77E07B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77E07B3E+238]


In [42]:
columns = ['score', 'review']
df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path = window_path)
for page_url in page_urls:
    
    # 상세보기 페이지에 접속합니다
    driver.get(page_url)
    time.sleep(2)
    
    # 첫 페이지 리뷰를 크롤링합니다
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contents_div = soup.find(name="div", attrs={"class":"cont_evaluation"})
    
    if contents_div is not None:
        # 별점을 가져옵니다.
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})

        # 리뷰를 가져옵니다.
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

        for rate, review in zip(rates, reviews):
            row = [rate.text[0], review.find(name="span").text]
            series = pd.Series(row, index=df.columns)
            df = df.append(series, ignore_index=True)

        # 2-5페이지의 리뷰를 크롤링합니다
        for button_num in range(2, 6):
            # 오류가 나는 경우(리뷰 페이지가 없는 경우), 수행하지 않습니다.
            try:
                another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
                another_reviews.click()
                time.sleep(2)

                # 페이지 리뷰를 크롤링합니다
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                contents_div = soup.find(name="div", attrs={"class":"cont_evaluation"})

                if contents_div is not None:
                    # 별점을 가져옵니다.
                    rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})

                    # 리뷰를 가져옵니다.
                    reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

                    for rate, review in zip(rates, reviews):
                        row = [rate.text[0], review.find(name="span").text]
                        series = pd.Series(row, index=df.columns)
                        df = df.append(series, ignore_index=True)
            except:
                break 
  
driver.close()


In [43]:
# 4점 이상의 리뷰는 긍정 리뷰, 3점 이하의 리뷰는 부정 리뷰로 평가합니다.
df['y'] = df['score'].apply(lambda x: 1 if float(x) > 2 else 0)
print(df.shape)
df.head()

(12, 3)


,score,review,y
0,3,뼈없는닭갈비와 치즈사리 폼과 맛 정말 미쳤다,1
1,3,,1
2,4,,1
3,3,,1
4,1,"대창 맛있음,,,",0
